In [53]:
from PIL import Image
import numpy as np
from numpy import *
from sklearn import preprocessing as prep
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import sklearn.cross_validation as crval
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import fetch_mldata
from scipy.interpolate import interp1d
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
import pylab
import csv
from sklearn.cross_validation import *
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier

from sklearn import neighbors

#LOADING DATA ..... 
X=[]
with open('x_tr.csv','r') as f:
    reader = csv.reader(f)
    for row1 in reader:
        X.append(row1)
y=[]
with open('y_tr.csv','r') as f:
    reader = csv.reader(f)
    for row1 in reader:
        y.append(row1)
        
X_test=[]
fp=open('class_test_in.csv','r')
trash=fp.readline()
#print trash
lines=fp.readlines()
print len(lines)
#print lines[0]

for line in lines:
    data=line.split(',')
    cleaned_data=data[1:len(data)]
    X_test.append(cleaned_data)



1963


In [52]:
print size(X)
print size(y)
X=np.asarray(X)
X=X.astype(np.float)
X=np.ravel(X)
X= X.reshape(1962, 265)

y=np.asarray(y)
y=np.ravel(y)
print shape(y)
#print X


X_test=np.asarray(X_test)
X_test=X_test.astype(np.float)
X_test=np.ravel(X_test)
print len(X_test)
X_test= X_test.reshape(1963, 265)

#NORMALIZING DATA
print(shape(X_test))    
selX=X_test
std_scal=prep.StandardScaler()
selX_scaled=std_scal.fit_transform(selX)
print('it had mean: ' +str(mean(selX))+' and std:'+ str(std(selX)))
print('NOW it has mean: '+str(mean(selX_scaled))+' and std: '+str(std(selX_scaled)))

X_test=selX_scaled

519930
1962
(1962,)
520195
(1963, 265)
it had mean: 93.7580887936 and std:24628.007464
NOW it has mean: -1.45060291886e-17 and std: 1.0


In [19]:
print('Lets mess up things a little bit: ')
selX_scaled=X
sely=y
indexes=[]
shuffly=y
shufflX=X
for i in range(len(sely)):
    indexes.append(i)
np.random.shuffle(indexes)
#print(indexes)
for i in range(len(indexes)):
    shuffly[i]=sely[i]
    shufflX[i, :]= selX_scaled[i,:]
print('total matrix of data:'+str(shape(shufflX)))
#print(shape(shuffly))

print('Divide et impera: creation of train/test/val sets:')

trainData, valData, trainLab, valLab = crval.train_test_split(shufflX, shuffly, test_size=0.2)

#trainData, valData, trainLab, valLab = crval.train_test_split(shufflX, shuffly, test_size=0.2)
#print(shape(otherData))
print('Done. Here the sizes: ')
print('Train: '+str(shape(trainData)))
print('Val: '+str(shape(valData)))



Lets mess up things a little bit: 
total matrix of data:(1962, 265)
Divide et impera: creation of train/test/val sets:
Done. Here the sizes: 
Train: (1569, 265)
Val: (393, 265)


In [20]:
#trainData, valData, trainLab, valLab = crval.train_test_split(shufflX, shuffly, test_size=0.2)
#alltrainData=trainData
#alltrainLab=trainLab
#print('Lets subsample the datasets, we do not have that much time')
#trainData, rubbishData, trainLab, rubbishLab= crval.train_test_split(trainData, trainLab, test_size=0.9)
#valData, rubbishData, valLab, rubbishLab= crval.train_test_split(valData, valLab, test_size=0.9)
#print('Train: '+str(shape(trainData)))
#print('Test: '+str(shape(testData)))
#print('Val: '+str(shape(valData)))
#countall(trainLab)

print('Time to grid-search!')
print('Range C: [2^-5,2^11]')
print('Range gamma: [2^-15, 2^3]')

accuracies=[]
best_choice_c=-5
best_choice_g=-15
old_accuracy=0
for ge in range(-15, 5, 2):
    g = pow(2,ge)
    for e in range(-5, 13, 2):
        c=pow(2,e)
        cls = SVC(C=c, kernel='rbf', gamma=g)
        cls.fit(trainData, trainLab)
        accuracy = cls.score(valData, valLab)
        #accuracy_kfold = cross_val_score(cls, valData, valLab, cv=5)
        #print accuracy_kfold
        #accuracy = np.sum(accuracy_kfold)
        #print np.sum(accuracy_kfold)
        if old_accuracy < accuracy:
            old_accuracy=accuracy
            best_choice_c=e
            best_choice_g=ge
        accuracies.append(accuracy)
        #plt.scatter(2**e,accuracy,color='c')
        print('Accuracy for c= 2^'+ str(e) + ' gamma= 2^'+str(ge)+':   ' + str(accuracy*100) +'%')
print('We will use c= 2^'+str(best_choice_c)+' and gamma= 2^'+str(best_choice_g)+'   '+str(old_accuracy))

c=pow(2,best_choice_c)
g=pow(2,best_choice_g)
cls = SVC(C=c, kernel='rbf', gamma=g)
cls.fit(alltrainData, alltrainLab)
#accuracy = cls.score(testData, testLab)
print('Getting back to the whole datasets: ')
print('Training with the c and gamma best performing values found')
#print ('Accuracy: '+str(accuracy*100)+'%')
#LOAD TEST SET, Compute predictions, save them on a csv

Time to grid-search!
Range C: [2^-5,2^11]
Range gamma: [2^-15, 2^3]
Accuracy for c= 2^-5 gamma= 2^-15:   47.8371501272%
Accuracy for c= 2^-3 gamma= 2^-15:   47.8371501272%
Accuracy for c= 2^-1 gamma= 2^-15:   47.8371501272%
Accuracy for c= 2^1 gamma= 2^-15:   54.1984732824%
Accuracy for c= 2^3 gamma= 2^-15:   54.9618320611%
Accuracy for c= 2^5 gamma= 2^-15:   55.9796437659%
Accuracy for c= 2^7 gamma= 2^-15:   56.2340966921%
Accuracy for c= 2^9 gamma= 2^-15:   57.5063613232%
Accuracy for c= 2^11 gamma= 2^-15:   62.8498727735%
Accuracy for c= 2^-5 gamma= 2^-13:   47.8371501272%
Accuracy for c= 2^-3 gamma= 2^-13:   47.8371501272%
Accuracy for c= 2^-1 gamma= 2^-13:   54.1984732824%
Accuracy for c= 2^1 gamma= 2^-13:   54.1984732824%
Accuracy for c= 2^3 gamma= 2^-13:   55.9796437659%
Accuracy for c= 2^5 gamma= 2^-13:   59.0330788804%
Accuracy for c= 2^7 gamma= 2^-13:   63.6132315522%
Accuracy for c= 2^9 gamma= 2^-13:   65.1399491094%
Accuracy for c= 2^11 gamma= 2^-13:   67.4300254453%
Accura

KeyboardInterrupt: 

In [11]:
#from sklearn.metrics import fbeta_score, make_scorer
#ftwo_scorer = make_scorer(fbeta_score, beta=2)
#from sklearn.cross_validation import *
#scores = cross_val_score(cls, alltrainData, alltrainLab, cv=5)
#print scores

print('Time to grid-search!')
print('Range C: [2^-5,2^11]')
print('Range gamma: [2^-15, 2^3]')

accuracies=[]
best_choice_c=-5
best_choice_g=-15
old_accuracy=0
order=2
for ge in range(2, 6, 1):
    g = pow(2,ge)
    ge=order
    for e in range(-1, 5, 1):
        c=pow(2,e)
        cls = SVC(C=c, kernel='poly', degree=ge)
        cls.fit(trainData, trainLab)
        accuracy = cls.score(valData, valLab)
        #accuracy_kfold = cross_val_score(cls, valData, valLab, cv=5)
        #print accuracy_kfold
        #accuracy = np.sum(accuracy_kfold)
        #print np.sum(accuracy_kfold)
        if old_accuracy < accuracy:
            old_accuracy=accuracy
            best_choice_c=e
            best_choice_g=ge
        accuracies.append(accuracy)
        #plt.scatter(2**e,accuracy,color='c')
        print('Accuracy for c= 2^'+ str(e) + ' gamma= 2^'+str(ge)+':   ' + str(accuracy*100) +'%')
print('We will use c= 2^'+str(best_choice_c)+' and gamma= 2^'+str(best_choice_g)+'   '+str(old_accuracy))

c=pow(2,best_choice_c)
g=pow(2,best_choice_g)
cls = SVC(C=c, kernel='rbf', gamma=g)
cls.fit(alltrainData, alltrainLab)
#accuracy = cls.score(testData, testLab)
print('Getting back to the whole datasets: ')
print('Training with the c and gamma best performing values found')
#print ('Accuracy: '+str(accuracy*100)+'%')
#LOAD TEST SET, Compute predictions, save them on a csv






Time to grid-search!
Range C: [2^-5,2^11]
Range gamma: [2^-15, 2^3]


NameError: name 'trainData' is not defined

In [5]:
C=pow(2,1)
g=pow(2,-7)
cls = SVC(C=C, kernel='rbf', gamma=g)
cls.fit(trainData, trainLab)
accuracy = cls.predict(X_test)


NameError: name 'trainData' is not defined

In [6]:
fp=open('answers.txt','w')
fp.write('Point_ID,Output\n')
i=1
for a in accuracy:
    fp.write(str(i)+','+a+'\n')
    i+=1
fp.close()

NameError: name 'accuracy' is not defined

In [13]:
print('Range C: [2^5,2^11]')
best_choice=-5
old_accuracy = 0
for e in range(5, 11, 2):
    c=pow(2,e)
    #print(old_accuracy)
    cls = LinearSVC(C=c)
    cls.fit(trainData, trainLab)
    accuracy = cls.score(valData, valLab)
    if old_accuracy<accuracy:
        old_accuracy=accuracy
        best_choice=e
    print('Accuracy for c= 2^'+ str(e) + ' :   ' + str(accuracy*100) +'%')
print('Best choice, at this moment: 2^' +str(best_choice))
print('It seems that c= 2^'+str(best_choice)+' works best, maybe we can try shifting the interval a little bit..' )
left=best_choice-5
right=best_choice+5
print('Range C: [2^'+str(left)+',2^'+str(right)+']')
i=0
accuracies=[0, 0, 0, 0, 0, 0, 0, 0, 0]
x = np.linspace(-15, 3, num=9, endpoint=True)
#print(shape(x))
#print(shape(accuracies))
for e in range(best_choice-5, best_choice+5, 2):
    c=pow(2,e)
    #print(c)
    cls = LinearSVC(C=c)
    cls.fit(trainData, trainLab)
    accuracy = cls.score(valData, valLab)
    #accuracy_kfold = cross_val_score(cls, valData, valLab, cv=5)
    print accuracy_kfold
    #print np.norm(accuracy_kfold)
    if old_accuracy < accuracy:
        old_accuracy=accuracy
        best_choice=e
    accuracies[i]=accuracy
    i+=1
    #plt.scatter(2**e,accuracy,color='c')
    print('Accuracy for c= 2^'+ str(e) + ' :   ' + str(accuracy*100) +'%')
print('Yes! Our intuition was good. We will use c= 2^'+str(best_choice))

#xnew = np.linspace(best_choice-5, best_choice-5, num=9, endpoint=True)
#f=interp1d(x,accuracies)
#plt.plot(xnew, f(xnew), '-')
#plt.grid(True)
#plt.show()

#c=pow(2,best_choice)
#cls = LinearSVC(C=c)
#cls.fit(trainData, trainLab)
#accuracy = cls.score(testData, testLab)
#print('Testing now the SVM with c= 2^'+str(best_choice)+' on testingDataset')
#print('Accuracy: '+ str(accuracy*100) +'%')

Range C: [2^5,2^11]


NameError: name 'trainData' is not defined

In [24]:
GPclassifier= GaussianProcessClassifier(1.0 * RBF(1.0), warm_start=True)
GPclassier.fit(trainData, trainLab)
accuracy = GPclassifier.score(valData, valLab)

NameError: name 'gpc' is not defined

In [29]:
#DECISION TREES

clf = tree.DecisionTreeClassifier()
clf = clf.fit(trainData, trainLab)
accuracy = clf.score(valData, valLab)
print accuracy

0.613231552163


In [31]:
#GRADIENT BOOSTING TREES

clf = GradientBoostingClassifier()
clf = clf.fit(trainData, trainLab)
accuracy = clf.score(valData, valLab)
print accuracy

0.648854961832


In [54]:
#RADIUS NEAREST NEIGHBORS 
n_neighbors=14
weights='distance' #otherwise 'uniform'
clf = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
clf.fit(trainData, trainLab)
accuracy = clf.predict(X_test)
#print accuracy
#accuracy valSet 77.86 -- bingo!


In [55]:
fp=open('answers.txt','w')
fp.write('Point_ID,Output\n')
i=1
for a in accuracy:
    fp.write(str(i)+','+a+'\n')
    i+=1
fp.close()